<a href="https://colab.research.google.com/github/AngelTroncoso/AngelTroncoso/blob/main/Copia_de_Streamlit2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance
!pip install ta
!pip install plotly
!pip install streamlit
!pip install streamlit-github-page
!streamlit run app.py

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=cedf8e7dc8d010ce4f1a43bf774371947bf98a5fcbe5afb9c002adc2bb5a49ca
  Stored in directory: /root/.cache/pip/wheels/5f/67/4f/8a9f252836e053e532c6587a3230bc72a4deb16b03a829610b
Successfully built ta
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
ERROR: Could not find a version that satisfies the requirement streamlit-github-page (from versions: none)
ERROR: No matching distributi

In [ ]:
import streamlit as st
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import ta
import plotly.express as px

# Botón para elegir color de fondo de la página
st.subheader('Visualización de Acciones del S&P 500')
color_pagina = st.color_picker('Selecciona un color para la página')

# Aplicar color de fondo a la página
st.markdown(f"""
    <style>
    .main {{
        background-color: {color_pagina};
    }}
    </style>
""", unsafe_allow_html=True)

# Título de la aplicación
st.title('Selecciona tres Acciones y Grafica, Compara y Descarga')

# Descripción
st.write('Esta aplicación te permite visualizar los datos históricos de las principales acciones del S&P 500.')

# Lista de símbolos del S&P 500 (puedes agregar más)
sp500_symbols = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'FB', 'JPM', 'JNJ', 'V', 'NVDA']

# Crear un selector en Streamlit para elegir una acción del S&P 500
selected_symbol = st.selectbox('Selecciona una acción del S&P 500', sp500_symbols)

# Crear dos selectores más para elegir acciones adicionales
selected_symbol2 = st.selectbox('Selecciona una segunda acción del S&P 500', sp500_symbols)
selected_symbol3 = st.selectbox('Selecciona una tercera acción del S&P 500', sp500_symbols)

# Selección de rango de fechas
start_date = st.date_input('Fecha de inicio', value=pd.to_datetime('2020-01-01'))
end_date = st.date_input('Fecha de fin', value=pd.to_datetime('today'))

# Botón para cargar datos y visualizar
if st.button('Mostrar gráfico'):
    # Descargar datos de Yahoo Finance
    data = yf.download([selected_symbol, selected_symbol2, selected_symbol3], start=start_date, end=end_date)

    if not data.empty:
        # Mostrar gráfico del precio de cierre
        st.subheader(f'Datos históricos de {selected_symbol}, {selected_symbol2} y {selected_symbol3}')
        st.line_chart(data['Close'])

        # Mostrar gráfico de candelas
        st.subheader('Gráfico de candelas')
        fig = go.Figure(data=[go.Candlestick(x=data.index,
                                            open=data['Open'][selected_symbol],
                                            high=data['High'][selected_symbol],
                                            low=data['Low'][selected_symbol],
                                            close=data['Close'][selected_symbol],
                                            name=selected_symbol),
                                    go.Candlestick(x=data.index,
                                                    open=data['Open'][selected_symbol2],
                                                    high=data['High'][selected_symbol2],
                                                    low=data['Low'][selected_symbol2],
                                                    close=data['Close'][selected_symbol2],
                                                    name=selected_symbol2),
                                    go.Candlestick(x=data.index,
                                                    open=data['Open'][selected_symbol3],
                                                    high=data['High'][selected_symbol3],
                                                    low=data['Low'][selected_symbol3],
                                                    close=data['Close'][selected_symbol3],
                                                    name=selected_symbol3)])
        fig.update_layout(title='Gráfico de candelas', xaxis_title='Fecha', yaxis_title='Precio')
        st.plotly_chart(fig)

        # Mostrar recomendaciones de análisis bursátiles financieros
        st.subheader('Recomendaciones de análisis bursátiles financieros')

        # Calcula la media móvil de 50 días
        data['MA50'] = data['Close'][selected_symbol].rolling(window=50).mean()

        # Calcula la media móvil de 200 días
        data['MA200'] = data['Close'][selected_symbol].rolling(window=200).mean()

        # Compara las medias móviles para determinar la tendencia
        if data['MA50'].iloc[-1] > data['MA200'].iloc[-1]:
            st.write('La tendencia es alcista')
        elif data['MA50'].iloc[-1] < data['MA200'].iloc[-1]:
            st.write('La tendencia es bajista')
        else:
            st.write('La tendencia es neutra')

        # Calcula el RSI (Relative Strength Index)
        data['RSI'] = ta.momentum.RSIIndicator(data['Close'][selected_symbol], window=14).rsi()

        # Compara el RSI para determinar si la acción está sobrecomprada o sobre vendida
        if data['RSI'].iloc[-1] > 70:
             st.write('La acción está sobrecomprada')
        elif data['RSI'].iloc[-1] < 30:
            st.write('La acción está sobre vendida')
        else:
            st.write('La acción está en un nivel neutral')

        # Opción para descargar datos
        st.subheader('Descargar datos')
        @st.cache
        def convert_df(df):
            return df.to_csv().encode('utf-8')
        csv = convert_df(data)
        st.download_button(
            label="Descargar datos como CSV",
            data=csv,
            file_name='datos.csv',
            mime='text/csv',
        )
    else:
        st.error('No se encontraron datos para este rango de fechas.')

    setup(
    name='tu_aplicacion',
    version='1.0',
    install_requires=[
        'yfinance'
    ]
)


2024-09-10 19:20:43.043 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-10 19:20:43.048 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-10 19:20:43.051 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-10 19:20:43.052 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-10 19:20:43.054 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-10 19:20:43.056 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-10 19:20:43.057 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-10 19:20:43.059 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [ ]:
!streamlit run Streamlit2024.ipynb

Usage: streamlit run [OPTIONS] TARGET [ARGS]...
Try 'streamlit run --help' for help.

Error: Streamlit requires raw Python (.py) files, not .ipynb.
For more information, please see https://docs.streamlit.io
